# iCompress - Compression based on eye tracking behavior

iCompress features image region-based compression made possible by extracting eye fixation regions from fixation data. <br />
iCompress transforms the eye fixation data to Fixation Density Maps (FDMs) and utilizes them to determine the image regions that should and should not be compressed. <br />
Encoding and decoding of an image is based on the JPEG compression algorithm.
More specifically, iCompress goes through similar steps to encode an image, but differs from it by using weighted quantization tables with respect to the degree of fixation found in the FDMs. <br />
Furthermore, iCompress leaves regions-of-interest (i.e. high degree of fixation) in tact and does not compress them. <br />
iCompress saves its files in <code>.icy</code> format.

This python notebook demonstrates the application of the algorithm on both images and videos with fixation data.
Throughout this notebook several functions (with possible small modifcations) from the CS4065 Multimedia Search and Recommendation Course Systems Lab 1 were used and accredited in line as such.

**To get started, please ensure that all the dependencies are installed!** <br />

## Dependencies:
* matplot-lib: https://matplotlib.org/users/installing.html
* OpenCV2: https://gist.github.com/arthurbeggs/06df46af94af7f261513934e56103b30
* scikit-image: http://scikit-image.org/download

## References:
* [1] H. Nemoto, P. Hanhart, P. Korshunov, and T. Ebrahimi, "Ultra-Eye: UHD and HD images eye tracking dataset." In International Workshop on Quality of Multimedia Experience (QoMEX), September 2014
* [2] T. Vigier, J. Rousseau, M. Perreira Da Silva, and P. Le Callet, “A new HD and UHD video eye tracking dataset.” In 7th ACM Multimedia Systems Conference (MMSys), May 2016.

## Setup imports
We first require to import all the necessary dependencies.

In [ ]:
%matplotlib inline

import math
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import cv2
import copy
import csv
from Queue import PriorityQueue
import subprocess as sp
import struct
import collections
import skimage.measure
import os

## Defining helper functions
These helper functions were taken from [cvtools.py](https://raw.githubusercontent.com/mmc-tudelft/cs4065/master/cvtools.py) from the MMR repository and slightly modified.

In [ ]:
def parse_numpy_image_size(image_size):
    """
    Usage:
    height, width, channels = parse_numpy_image_size(np.shape(image))
    """
    try:
        height, width, channels = image_size
    except:
        height, width = image_size
        channels = 1
    return height, width, channels

def format_image_size(image_size):
    """
    Usage:
    formatted_image_size = format_image_size(np.shape(image))
    """
    height, width, channels = parse_numpy_image_size(image_size)
    return '%d x %d (%d channels)' % (width, height, channels)

In [ ]:
_DEFAULT_IMAGE_FIGSIZE = (16, 16)
IMAGE_HEIGHT = 2160
IMAGE_WIDTH = 3840

def ipynb_show_cv2_image(image_in, title='', figsize=_DEFAULT_IMAGE_FIGSIZE, grayscale = False, save=False):
    _, _, channels = parse_numpy_image_size(np.shape(image_in))
    image = image_in.copy()
    if channels > 1:
        image[:, :, 0] = image_in[:, :, 2]
        image[:, :, 2] = image_in[:, :, 0]
    ipynb_show_image(image, title, figsize, grayscale, save)

def ipynb_show_image(image, title='', figsize=_DEFAULT_IMAGE_FIGSIZE, grayscale = False, save=False):
    fig, ax = plt.subplots(figsize=figsize)
    ax.set_title(title)
    if grayscale:
        plt.imshow(image, cmap = 'gray')
    else:
        plt.imshow(image)
    plt.axis('off')

    if save:
        plt.savefig(title + '.png')
    plt.show()
    return ax

## Reading in one of the images from the UltraEye dataset [1]
To test the algorithm for combining the fixation density maps (FDMs) with the original image and splitting it into macroblocks,
we can take an image from our UHD image dataset and its FDM.

In [ ]:
# First tryout of reading image
im = cv2.imread("ultraeye/C01_UHD.tif", cv2.IMREAD_UNCHANGED)
im_size = np.shape(im)

print 'Original image size: %s' % format_image_size(im_size)
ipynb_show_cv2_image(im, title = 'Source image: C01')

In [ ]:
# Set size for our macro block
MACROBLOCK_SIZE = 8

In [ ]:
# Plot one of our fixation density maps
im_fdm = cv2.imread("ultraeye/C01_FDM_UHD.tif", cv2.IMREAD_UNCHANGED)

im_size_fdm = np.shape(im_fdm)
print 'Original image size: %s' % format_image_size(im_size_fdm)

# Plot our fixation density map
ipynb_show_cv2_image(im_fdm, title = 'Fixation density map: C01', grayscale = True)

# Pre-process our FDM for selecting the more relevant sections
# May not be necessary for selecting the macroblocks in the end, but is nice for the visualization.
MAX_PIX_VAL = 255
THRESHOLD_SCALAR = 0.2
white_threshold = THRESHOLD_SCALAR*MAX_PIX_VAL
_, im_fdm_thresh = cv2.threshold(im_fdm, white_threshold, MAX_PIX_VAL, cv2.THRESH_BINARY)
ipynb_show_cv2_image(im_fdm_thresh, title = 'Thresholded fixation density map: C01', grayscale = True)

## Plot the combination of the thresholded FDM and original image.
From our previous (thresholded) FDM, we now know what regions are 'important'. These regions are available to us as pixel coordinates. <br />

In [ ]:
# Compute the pixel coordinates of a FDM
def computeFDMRegionCoordinates(fdm):
    coordinates = []
    for i in range(0, len(fdm)):
        coords = (fdm[i].flatten()[0], fdm[i].flatten()[1])
        coordinates.append(coords)
    return coordinates

# Draw the FDM region on top of an image.
# Input:
# - Image to draw FDM on top of
# - Coordinates of the FDM to draw
def drawFDMRegions(image, coordinates):
    for i in range(0, len(coordinates)):
        coords = (coordinates[i][0], coordinates[i][1])
        cv2.circle(image, coords, 1, (0, 0, 255), -1, cv2.CV_AA)

    ipynb_show_cv2_image(image)

Let's see what our FDM looks like on top of our original image.

In [ ]:
nonZero_fdm = cv2.findNonZero(im_fdm_thresh)
if nonZero_fdm is not None:
    region_fdm_image = cv2.imread("ultraeye/C01_UHD.tif", cv2.IMREAD_UNCHANGED)
    num_rows = nonZero_fdm.shape[0]/im_size_fdm[1]

    im_FDMcoords = computeFDMRegionCoordinates(nonZero_fdm)
    drawFDMRegions(region_fdm_image, im_FDMcoords)

What's left is to take the inverse and use those pixel coordinates and map them to the macro block coordinates.

In [ ]:
# Compute the inverted regions, so we can use the indices as selectors for our region based compression
inverted_region_fdm_image = cv2.imread("ultraeye/C01_UHD.tif", cv2.IMREAD_UNCHANGED)
inverted_fdm_thres = cv2.bitwise_not(im_fdm_thresh)
inverted_fdm = cv2.bitwise_not(im_fdm)

# Show the inverted FDM
ipynb_show_cv2_image(inverted_fdm, title = 'Inverted fixation density map: C01', grayscale = True)

# Show the inverted thresholded FDM
ipynb_show_cv2_image(inverted_fdm_thres, title = 'Inverted thresholded fixation density map: C01', grayscale = True)

# This code shows the inverted FDM on top of our original, but is rather slow and not needed for the actual compression
# nonZero_inverted_fdm = cv2.findNonZero(inverted_fdm_thres)

# inverted_im_FDMcoords = computeFDMRegionCoordinates(nonZero_inverted_fdm)
# # Draw the image with our inverted FDM regions once again
# drawFDMRegions(inverted_region_fdm_image, inverted_im_FDMcoords)

## Helper functions taken from the MMSR systems lab 1.
These are some functions to determine the macroblock coordinates given pixel coordinates and converting back

In [ ]:
def get_macroblock_coords((x, y), block_size = MACROBLOCK_SIZE):
    """Computes the macroblock coordinates of a pixel given the (square) block size."""
    def _macroblock_index(pos):
    # This is the core function that does the job for get_number_of_macroblocks().
        return pos/block_size
    return (_macroblock_index(x), _macroblock_index(y))

def get_number_of_macroblocks(image_size, block_size = MACROBLOCK_SIZE):
    """Returns the number of horizontal and vertical macroblocks."""
    return get_macroblock_coords((image_size[1], image_size[0]), block_size)

def macroblock_coords_to_pixel_vertices(macroblock_coords, block_size = MACROBLOCK_SIZE):
    """Maps a macroblock onto pixel vertex coordinates."""
    x0 = macroblock_coords[0]*block_size
    y0 = macroblock_coords[1]*block_size
    x1 = macroblock_coords[0]*block_size + block_size - 1
    y1 = macroblock_coords[1]*block_size + block_size - 1
    return (x0, y0), (x1, y1)

## Investigate macroblocks in our image
Taken from the taken from the MMSR systems lab 1, we inspect our image's macroblocks.

In [ ]:
print 'Image size: %s' % format_image_size(im_size)
print 'Number of macroblocks: %d horizontal and %d vertical' % get_number_of_macroblocks(im_size)

In [ ]:
# Taken from the MMSR systems lab 1.
# Parameters (explained below when used for the first time).
USE_ANTIALIASING_FILTER = True
FILTER_SIZE = (2, 2)
SUBSAMPLING_FACTORS = (2, 2)  # Vertical and horizontal. https://en.wikipedia.org/wiki/Chroma_subsampling
# These numbers mean that we skip X-1 vertical pixels in original image (step size), and skip Y-1 horizontal.
# So (1,2) means step size 1 for vertical, taking all pixels into account and 2 means halving horizontal pixel.
# We get therefore a 4:2:2 subsampling.
# For (2,2), we get 4:2:0 subsampling.
# (1,1) is 4:4:4 subsampling.
# (1,4) is 4:1:1
# (4,2) is 4:4:0

def subsample_chrominance_channels(image):
    # Let's convert the RGB image to the YUV space.
    # You will apply compression in such color space.
    image_YCrCb = cv2.cvtColor(image, cv2.cv.CV_BGR2YCrCb)

    # Anti-aliasing filter.
    # Before we subsample the Cr and Cb channels, we apply an averaging filter to avoid artifacts.
    # We use cv2.boxFilter() for this (see http://docs.opencv.org/2.4/modules/imgproc/doc/filtering.html#boxfilter).
    if USE_ANTIALIASING_FILTER:
        image_Cr_channel_filtered = cv2.boxFilter(image_YCrCb[:,:,1], ddepth=-1, ksize=FILTER_SIZE)
        image_Cb_channel_filtered = cv2.boxFilter(image_YCrCb[:,:,2], ddepth=-1, ksize=FILTER_SIZE)
    else:
        image_Cr_channel_filtered = image_YCrCb[:,:,1]
        image_Cb_channel_filtered = image_YCrCb[:,:,2]

    # Subsampling step: we will skip crominance values (similar to image resizing).
    # We use numpy array slicing (see http://docs.scipy.org/doc/numpy/reference/arrays.indexing.html).
    image_Cr_channel_subsampled = image_Cr_channel_filtered[
      ::SUBSAMPLING_FACTORS[0], ::SUBSAMPLING_FACTORS[1]]
    image_Cb_channel_subsampled = image_Cb_channel_filtered[
      ::SUBSAMPLING_FACTORS[0], ::SUBSAMPLING_FACTORS[1]]

    return [
        image_YCrCb[:,:,0],  # We don't toch the luminance channel.
        image_Cr_channel_subsampled,
        image_Cb_channel_subsampled
    ]

image_YCrCb_subsampled = subsample_chrominance_channels(im)

## Determining regions of our image to compress
To compress only the regions with respect to the FDM, we first divide our image into macroblocks. <br />
Then we create a boolean matrix indicating which macroblocks should be quantized later on. <br />
We give a 1, indicating that the macroblock should be quantized, iff the entire macroblock is found inside the coordinates.
Otherwise, when a single pixel coordinate inside a macroblock is not found, the macroblock should be left untouched.

In [ ]:
# Print our channel sizes
print 'Y size: ' + str(image_YCrCb_subsampled[0].shape)
print 'Cr size: ' + str(image_YCrCb_subsampled[1].shape)
print 'Cb size: ' + str(image_YCrCb_subsampled[2].shape)

In [ ]:
# Take in a channel and convert that channel into macroblocks
def computeSelectionMacroBlock(channel):
    channel_FDMsize = channel.shape

    num_macroblock = get_number_of_macroblocks(channel_FDMsize)
    print 'Shape macro block: ' + str(num_macroblock)
    macroblock_im_FDMcoords = np.zeros((num_macroblock[0], num_macroblock[1]))

    for i in range(0, channel_FDMsize[0], MACROBLOCK_SIZE):
        for j in range(0, channel_FDMsize[1], MACROBLOCK_SIZE):
            macroblock_im_FDMcoords[j/MACROBLOCK_SIZE, i/MACROBLOCK_SIZE] = np.amin(channel[i:i+MACROBLOCK_SIZE-1, j:j+MACROBLOCK_SIZE-1])
    return macroblock_im_FDMcoords

# Takes in a FDM and computes the regions which we need
def computeChannelRegions(FDM):
    # Leave Y alone
    Y_bool_inverted_regions = FDM.astype(bool)*1 # Make a boolean 0 and 1 matrix

    # Use subsampled boolean FDM for Cr and Cb
    Cr_bool_inverted_regions = Y_bool_inverted_regions[::SUBSAMPLING_FACTORS[0], ::SUBSAMPLING_FACTORS[1]]
    Cb_bool_inverted_regions = Y_bool_inverted_regions[::SUBSAMPLING_FACTORS[0], ::SUBSAMPLING_FACTORS[1]]

    Y_macro_blocks = computeSelectionMacroBlock(Y_bool_inverted_regions)
    Cr_macro_blocks = computeSelectionMacroBlock(Cr_bool_inverted_regions)
    Cb_macro_blocks = computeSelectionMacroBlock(Cb_bool_inverted_regions)
    return [Y_macro_blocks, Cr_macro_blocks, Cb_macro_blocks]

channelRegions = computeChannelRegions(inverted_fdm_thres)

In [ ]:
# Taken from the MMSR systems lab 1
# Luminance and chrominance quantization tables.
# (from Annex K of https://www.w3.org/Graphics/JPEG/itu-t81.pdf).
def parse_quantization_table(data):
    return np.array([[
      np.float64(cell) for cell in row.split(' ')] for row in (
          data.split('\n'))], dtype=np.float64)

# Quantization table to be used on the Y channel.
JPEG_STANDARD_LUMINANCE_QUANTIZATION_TABLE = parse_quantization_table(
"""16 11 10 16 24 40 51 61
12 12 14 19 26 58 60 55
14 13 16 24 40 57 69 56
14 17 22 29 51 87 80 62
18 22 37 56 68 109 103 77
24 35 55 64 81 104 113 92
49 64 78 87 103 121 120 101
72 92 95 98 112 100 103 99""")

# Quantization table to be used on the Cr and Cb channels.
JPEG_STANDARD_CHROMINANCE_QUANTIZATION_TABLE = parse_quantization_table(
"""17 18 24 47 99 99 99 99
18 21 26 66 99 99 99 99
24 26 56 99 99 99 99 99
47 66 99 99 99 99 99 99
99 99 99 99 99 99 99 99
99 99 99 99 99 99 99 99
99 99 99 99 99 99 99 99
99 99 99 99 99 99 99 99""")

## Weight our quantization tables with respect to the FDM
Since we thresholded all regions through a hard selector, we wish to retain that some areas that fall outside this threshold may still be more important than other areas.
To account for these regions, we use the values of the original FDM in those regions to weight our compression rate.

In [ ]:
luminance_quantization_table = copy.deepcopy(JPEG_STANDARD_LUMINANCE_QUANTIZATION_TABLE)
chrominance_quantization_table = copy.deepcopy(JPEG_STANDARD_CHROMINANCE_QUANTIZATION_TABLE)

print 'size new luminance quantization table: ' + str(luminance_quantization_table.shape)
print 'size new chrominance quantization table: ' + str(chrominance_quantization_table.shape)

COMPRESSION_RATE_LUMINANCE = 3 # The aggressiveness of the division during quantization, note that a rate of 1 still causes lossy compression!
COMPRESSION_RATE_CHROMINANCE = 1

# something with compression_rate*weight
luminance_quantization_table *= COMPRESSION_RATE_LUMINANCE
chrominance_quantization_table *= COMPRESSION_RATE_CHROMINANCE

print luminance_quantization_table
print chrominance_quantization_table

In [ ]:
# Taken from the MMSR systems lab 1
quantization_tables = [
    luminance_quantization_table,  # Y channel.
    chrominance_quantization_table,  # Cr channel.
    chrominance_quantization_table,  # Cb channel.
]

In [ ]:
# Taken from the MMSR systems lab 1
def quantizemacroblock_dct(macroblock, quantization_table):
    """
    Quantize DCT coefficients of a macroblock given a quantization table.

    DCT coefficients are real values.
    We first scale them (the larger the coefficients in the quantization table,
    the smaller will be the scaled values).
    Finally, we round off the scaled values in order to approximate each coefficient
    to an integer value (quantization process).

    The quantized DCT coefficients will be used with loss-less compression techniques
    in order to store them efficiently.
    """
    assert np.shape(macroblock) == np.shape(quantization_table), (
      np.shape(macroblock), np.shape(quantization_table))

    macroblock_dct = cv2.dct(macroblock)
    macroblock_dct = np.round(np.divide(macroblock_dct, quantization_table))

    quantized_macroblock_dct = macroblock_dct

    return quantized_macroblock_dct

In [ ]:
# Taken from the MMSR systems lab 1 and modified to incorporate regions from our FDMs.
# Quantize regions of an image
def quantize_image_YCrCb_dct(image_YCrCb_subsampled, scaled_quantization_tables, image_size, bln_verbose = True):
    if bln_verbose:
        def _vprint(what):
            print what
    else:
        def _vprint(what):
            pass
    _vprint('Quantizing DCT coefficients')

    # Initialization and parameters.
    image_YCrCb_dct_quantized = []
    block_size = MACROBLOCK_SIZE
    scaled_fdm = computeSelectionMacroBlock(inverted_fdm/float(np.max(inverted_fdm)))

    # Each channel is independently processed.
    for channel_index, (channel, quantization_table) in enumerate(
        zip(image_YCrCb_subsampled, scaled_quantization_tables)):

        # Channel properties.
        channel_size = np.shape(channel)
        number_of_horizontal_macroblocks, number_of_vertical_macroblocks = (
            get_number_of_macroblocks(channel_size, block_size))
        quantization_table_data_type = quantization_table.dtype

        _vprint('. channel #%d' % channel_index)
        _vprint('  channel size: %s' % format_image_size(channel_size))
        _vprint('  number of macroblocks: %d x %d' % (
            number_of_horizontal_macroblocks, number_of_vertical_macroblocks))

        # Initialize quantized channel matrix.
        channel_dct_quantized = np.zeros(channel_size, dtype=quantization_table_data_type)

        # Our boolean matrix to indicate which macroblocks should be quantized.
        macroblock_selector = channelRegions[channel_index]

        # Each macroblock is independently quantized.
        for row in range(number_of_vertical_macroblocks):
            for column in range(number_of_horizontal_macroblocks):
                # Extract the macroblock from the channel matrix.
                (left_top, right_bottom) = macroblock_coords_to_pixel_vertices((column, row), block_size)
                macroblock = channel[left_top[1]:right_bottom[1]+1, left_top[0]:right_bottom[0]+1]

                if macroblock_selector[column, row] == 1:
                    # Get the FDM scalar
                    FDM_scalar = scaled_fdm[column, row]
                    # Compute and quantize DCT coefficients.
                    macroblock_dct_quantized = quantizemacroblock_dct(
                        macroblock.astype(quantization_table_data_type), FDM_scalar*quantization_table)
                else:
                    macroblock_dct_quantized = macroblock
                # Copy macroblock data.
                channel_dct_quantized[
                    left_top[1]:right_bottom[1]+1, left_top[0]:right_bottom[0]+1] = (
                        macroblock_dct_quantized)

        _vprint('  quantized DCT coefficients matrix size: %d x %d' % np.shape(channel_dct_quantized))
        image_YCrCb_dct_quantized.append(channel_dct_quantized)

    return image_YCrCb_dct_quantized

In [ ]:
# Taken from the MMSR systems lab 1
# We quantize our image and obtain the Discrete Cosine Transform Coefficients.
image_YCrCb_dct_quantized = quantize_image_YCrCb_dct(
    image_YCrCb_subsampled, quantization_tables, im_size)

## Entropy encoding
Now we have quantified our image, we are going to compress it with the Huffman coding.
It encodes every symbol which occur in the image to a prefix code based on their frequency.

## Run-length encoding with zig-zag
When we look at our quantized image, we can see that there are many zeros in our every channel. To reduce the number of bits necessary to store our data, we use run-length encoding(RLE).  RLE counts the amount of consecutive characters and stores the number with the corresponding symbol. For JPEG they changed it a bit by only storing the consecutive amount of zeros, without explicitly indicating it is for the zeros. Furthermore they store the non-zero number that occurred after those zeros. JPEG also store the amount of bytes necessary to store the non-zero number, but for our implantation we do not need it, so we omitted it.
To optimally find a long chain of zeros, they also zigzag through the image. They zigzag through a macroblock, because of the quantization table, where the least compression happens in the top left corner. When moving further to the bottom right corner, the values are getting compressed more and more zeros are visible.
Lastly, because most macroblocks, have only 1 non-zero number which is the top left one, we will only store the value of the first value of the first macroblock. For the blocks after this one, we store the difference between the previous block.
So our data will be stored in pairs, like this: amount of zeros, non-zero value.

In [ ]:
# Because we only have 8x8 MACROBLOCKS we hardcode the zigzag.
j_zig = [0,1,0,0,1,2,3,2,1,0,0,1,2,3,4,5,4,3,2,1,0,0,1,2,3,4,5,6,7,6,5,4,3,2,1,0,1,2,3,4,5,6,7,7,6,5,4,3,2,3,4,5,6,7,7,6,5,4,5,6,7,7,6,7]
i_zig = [0,0,1,2,1,0,0,1,2,3,4,3,2,1,0,0,1,2,3,4,5,6,5,4,3,2,1,0,0,1,2,3,4,5,6,7,7,6,5,4,3,2,1,2,3,4,5,6,7,7,6,5,4,3,4,5,6,7,7,6,5,6,7,7]

#Zig-zag through a size macroblock by macroblock array and encode it with run-length encoding (RLE)
def zigzag_macroblock(quant_arr_macroblock_size, quant_table, count_zeros, DC, is_first):
    for zig_all in range(MACROBLOCK_SIZE*MACROBLOCK_SIZE):
        quant_arr_value = quant_arr_macroblock_size[i_zig[zig_all]][j_zig[zig_all]]
        if zig_all ==  0:
            temp_DC_store = quant_arr_value
            quant_arr_value = quant_arr_value - DC #Remove '- DC' if you want to remove the calculation between first elements of blocks
            if is_first == True:
                DC = quant_arr_value
            else:
                DC = temp_DC_store
            is_first = False
        if quant_arr_value != 0:
            quant_table.append(count_zeros)
            quant_table.append(quant_arr_value)
            count_zeros = 0
        else:
            count_zeros += 1
    return count_zeros, quant_table, DC, is_first

def encode_zigzag(quant_arr):
    #print np.asarray(quant_arr)[0].shape
    #print quant_arr[0].shape
    zigzag_quant = []
    trailing_zeros = 0
    for channel in range(0,len(quant_arr)):
        first_round = True
        trailing_zeros = 0
        DC_coefficient = 0 #Start DC value
        zigzag_quant.append([])
        for i in range(0, quant_arr[channel].shape[0], MACROBLOCK_SIZE):
            for j in range(0, quant_arr[channel].shape[1], MACROBLOCK_SIZE):
                trailing_zeros, zigzag_quant[channel], DC_coefficient, first_round = zigzag_macroblock(quant_arr[channel][i:i+MACROBLOCK_SIZE,j:j+MACROBLOCK_SIZE], zigzag_quant[channel], trailing_zeros, DC_coefficient,first_round)
        zigzag_quant[channel].append(0) #End with 2 zeros, so we know there are only trailing zeros left.
        zigzag_quant[channel].append(0)
    return zigzag_quant

In [ ]:
#Reference: https://stackoverflow.com/questions/11587044/how-can-i-create-a-tree-for-huffman-encoding-and-decoding

# The Huffman tree is stored inside this object.
class HuffmanNode(object):
    def __init__(self, left=None, right=None):
        self.left = left
        self.right = right
    def children(self):
        return((self.left, self.right))

# We build a Huffman tree based on the frequency of each symbol.
def create_tree(frequencies):
    p = PriorityQueue()
    for value in frequencies:    # 1. Create a leaf node for each symbol
        p.put(value)             #    and add it to the priority queue
    while p.qsize() > 1:         # 2. While there is more than one node
        l, r = p.get(), p.get()  # 2a. remove two highest nodes
        node = HuffmanNode(l, r) # 2b. create internal node with children
        p.put((l[0]+r[0], node)) # 2c. add new node to queue
    return p.get()               # 3. tree is complete - return root node

# Recursively walk the tree down to the leaves,
#   assigning a code value to each symbol
def walk_tree(node, prefix="", code={}):
    if(isinstance(node[1], HuffmanNode)):
        walk_tree(node[1].children()[0],prefix=prefix+'0')
        walk_tree(node[1].children()[1],prefix=prefix+'1')
    else:
        code.update({node[1]:prefix})
    return(code)

#Create a Huffman table for each array element in 'quant_arr'
def huffman_encoding(quant_arr):
    huffman_table = []
    for i in range(0,len(quant_arr)): #Loop all array elements
        huffman_table.append([])
        unique, counts = np.unique(quant_arr[i], return_counts=True)
        store_freq = list(zip(counts,unique))
        node = create_tree(store_freq)
        code = walk_tree(node) #Huffman codes
        for j in sorted(store_freq, reverse=True): #Sort by frequency
            #print(j[1], '{:6.2f}'.format(j[0]), code[j[1]]) #Print the table
            huffman_table[i].append((j[1], code[j[1]]))
    return huffman_table

In [ ]:
zigzag_quant = encode_zigzag(image_YCrCb_dct_quantized)
huffman_quant_table = huffman_encoding(zigzag_quant)
#huffman_quant_table = huffman_encoding(image_YCrCb_dct_quantized)
print np.asarray(huffman_quant_table[0])

## Canonical Huffman codebook: Encoding Huffman code into a Canonical Huffman tree
This step transforms our Huffman code we computed over our DCCs into a Canonical Huffman tree. <br />
This is done by a few steps, as follows: <br />
* First, the bit-length of all the Huffman code w.r.t. its symbol is computed.
* Thereafter, the Huffman code is sorted in increasing order based on the **bit-length** of all the symbols.
* Finally, the amount of symbols with **equal** bit-length are counted.

In [ ]:
def computeCanonicalCode(huffman_quant_table, bln_verbose = True):
    if bln_verbose:
        def _vprint(what):
            print what
    else:
        def _vprint(what):
            pass

    num_channels = len(huffman_quant_table)

    canonicalCodebook = []

    # We compute the canonical code per channel
    for i in range(0, num_channels):
        _vprint('**** Begin Huffman code: ' + str(i+1) + ' ****')
        _vprint(huffman_quant_table[i])
        _vprint('-----------------')

        # Sort our code to ascending order for bit-length of symbols
        num_codebook_symbols = np.asarray(huffman_quant_table[i]).shape[0]
        chm_BitLengthFreq = np.zeros((num_codebook_symbols, 2))
        for j in range(0, len(chm_BitLengthFreq)):
            chm_BitLengthFreq[j][0] = huffman_quant_table[i][j][0]
            chm_BitLengthFreq[j][1] = len(huffman_quant_table[i][j][1])

        _vprint('Canonical Huffman code bit length frequency')
        _vprint(chm_BitLengthFreq)
        _vprint('-----------------')

        chm_sortedByBitLength = np.asarray(sorted(chm_BitLengthFreq, key=lambda x: x[1]))

        _vprint('Sorted canonical Huffman code by bit length frequency')
        _vprint(chm_sortedByBitLength)
        _vprint('-----------------')

        # Compute bit length sizes, insert a 0 when a bit length size is 'missing'.
        max_bit_length = int(max(chm_sortedByBitLength[:, 1]))
        bitLengthSorted = np.zeros((max_bit_length, 1), dtype = int)

        for i in range(0, len(chm_sortedByBitLength)):
            bitLengthSorted[int(chm_sortedByBitLength[i][1])-1] += 1

        bitLengthSorted = bitLengthSorted.flatten()
        _vprint('Compute bit length sizes')
        _vprint(bitLengthSorted)
        _vprint('-----------------')

        all_symbols = chm_sortedByBitLength[:, 0]
        chm_codebook = [bitLengthSorted.T, all_symbols]
        canonicalCodebook.append(chm_codebook)
        _vprint('Canonical bit length sorted codebook')
        _vprint(np.asarray(chm_codebook))
        _vprint('-----------------')
    return np.asarray(canonicalCodebook)

In [ ]:
canonicalCodebook = computeCanonicalCode(huffman_quant_table, bln_verbose=False)
print canonicalCodebook[0]

## Decoding the canonical Huffman codebook
We decode the canonical Huffman codebook to derive the encoded canonical Huffman code with its mapping to original symbol.

In [ ]:
# Construct canonical Huffman code
def decodeCanonicalCodebook(chm_codebook, bln_verbose = True):
    if bln_verbose:
        def _vprint(what):
            print what
    else:
        def _vprint(what):
            pass

    lookup_table = []
    for k in range(0, len(chm_codebook)):
        symbols = chm_codebook[k][1]
        freq = [int(float(x)) for x in chm_codebook[k][0]]

        init_bit_index = 0
        # Find the index of the least amount of bits we need for any symbol
        for i in range(0, len(freq)):
            if freq[i] > 0:
                init_bit_index = i+1
                break

        # Reconstruct our symbol frequencies
        bit_length = []
        for i in range(0, len(freq)):
            if freq[i] > 0: # Code with that bit length is *used* in actual codebook
                for j in range(0, freq[i]):
                    bit_length.append(i+1)

        _vprint('Reconstructed symbol frequencies to use in sequential order')
        _vprint(bit_length)
        _vprint('-----------------')

        _vprint('Decoded codebook -> Canonical Huffman code')
        ch_code = [] # Save our huffman code here

        # Construct our first code
        trailing_bits = 2 + init_bit_index # 2 characters reserved for characters indicating binary format '0b'
        code = 0b0 # Starting code in pure binary form (note: that this is without any trailing 0 used for computation)
        _vprint((symbols[0], format(0, '#0'+ str(trailing_bits) + 'b')[2:]))

        ch_code.append([symbols[0], format(0, '#0'+ str(trailing_bits) + 'b')[2:]]) # Store first code

        # Construct the rest of our canonical code (pseudocode from [wikipedia](https://en.wikipedia.org/wiki/Canonical_Huffman_code#Pseudo_code))
        for i in range(1, len(symbols)):
            bit_length_next = bit_length[i]
            current_bit_length = (code + 1).bit_length()
            code = (code + 1) << abs(bit_length_next - current_bit_length)
            ch_code.append([symbols[i], str(bin(code)[2:])])
            _vprint((symbols[i], bin(code)[2:]))

        _vprint('-----------------')
        _vprint(('Codebook:', ch_code))
        lookup_table.append(ch_code)
    return np.asarray(lookup_table)

In [ ]:
canonicalHuffmanCode = decodeCanonicalCodebook(canonicalCodebook, bln_verbose=False)

## Encoding the image with Canonical Huffman code
We now have our lookup table and a tree to derive the lookup table ready! <br />
This means all that remains is to actually encode our DCTCs of the quantizated image. <br />
Then all we need to save is the tree (i.e. the codebook) **and** our encoded DCTCs.

We put our lookup table in the form of a dictionary to make it more easy to have a direct mapping from our code to symbol.

In [ ]:
# Make a numpy array into a dictionary for easy lookup
def convertToLookupTable(canonicalHuffmanCodebook):
    lookup_table = []
    # Go through Y, Cr, Cb
    for i in range(0, len(canonicalHuffmanCodebook)):
        channel_mapping = {}

        # Go through all the symbol:code pairs
        for j in range(0, len(canonicalHuffmanCodebook[i])):
            symbol = canonicalHuffmanCodebook[i][j][0]
            code = canonicalHuffmanCodebook[i][j][1]
            channel_mapping[symbol] = code
        lookup_table.append(channel_mapping)

    return lookup_table

lookupTable = convertToLookupTable(canonicalHuffmanCode)

## Encode the RLE DCTCs
We now encode the Run-Length Encoded DCTCs with our lookup table.
This should result in a very compact representation of our DCTCs.

In [ ]:
# Encodes a quantized zig-zag image with Canonical Huffman code
def encodeDCTQuantizedZigZag(image_YCrCb_dct_quantized, lookupTable):
    num_channels = np.asarray(image_YCrCb_dct_quantized).shape[0]

    chm_encoded_dct_quantized = []
    for i in range(0, num_channels):
        encodedChannel = []
        lookupTableChannel = lookupTable[i] # Get the lookup table matching the channel
        for j in range(0, len(image_YCrCb_dct_quantized[i])):
            DCTCValue = image_YCrCb_dct_quantized[i][j]
            encodedDCTC = lookupTableChannel[DCTCValue]
            encodedChannel.append(encodedDCTC)
        chm_encoded_dct_quantized.append(np.asarray(encodedChannel))
    chm_encoded_dct_quantized = np.asarray(chm_encoded_dct_quantized)

    return chm_encoded_dct_quantized

chm_encoded_dct_quantized = encodeDCTQuantizedZigZag(zigzag_quant, lookupTable)

## Writing to file
We are now ready to save the encoded image and its codebook to file.
We pack the data by converting all data to hexadecimal representation.

In [ ]:
def encodeCodeBook(FILEIN, canonicalCodebook, bln_verbose = True):
    if bln_verbose:
        def _vprint(what):
            print what
    else:
        def _vprint(what):
            pass

    _vprint('Encoding codebook raw format...')
    # Now append the codebook to the bitstream: we write how much numbers there are for the Y and Cr channels
    codebook_bitstream = ''
    for i in range(0, len(canonicalCodebook)):
        codebook = canonicalCodebook[i]
        freqs = codebook[0]
        symbols = codebook[1]
        freq_size = len(freqs)
        symbol_size = len(symbols)

        # Add the frequencies
        for j in range(0, freq_size):
            codebook_bitstream += str(int(freqs[j]))
            if j < freq_size-1:
                codebook_bitstream += ','
        codebook_bitstream += '|'

        # Add the symbols
        for m in range(0, symbol_size):
            codebook_bitstream += str(int(symbols[m]))
            if m < symbol_size-1:
                codebook_bitstream += ','

        codebook_bitstream += '|'
    _vprint(codebook_bitstream)

    f = open(FILEIN, 'w')
    f.write(codebook_bitstream)
    print len(codebook_bitstream)
    f.close()
    _vprint('Finished encoding codebook raw format...')

def encodeImageRaw(FILEIN, chm_encoded_dct_quantized, bln_verbose = True):
    if bln_verbose:
        def _vprint(what):
            print what
    else:
        def _vprint(what):
            pass

    _vprint('Encoding bitstream raw format...')
    
    # Write all content to file as raw output
    num_channels = chm_encoded_dct_quantized.shape[0]
    f = open(FILEIN, 'a')

    for i in range(0, num_channels):
        channel_encoded_dct_quantized = chm_encoded_dct_quantized[i].flatten()
        channel_encoded_dct_quantized_size = len(channel_encoded_dct_quantized)
        for j in range(0, channel_encoded_dct_quantized_size):
            code = channel_encoded_dct_quantized[j]
            f.write(code)
            
    f.close()
    _vprint('Finished encoding bitstream raw format...')

FILEIN = 'encodedimage.raw'

encodeCodeBook(FILEIN, canonicalCodebook)
encodeImageRaw(FILEIN, chm_encoded_dct_quantized)

In [ ]:
FILEOUT = 'encodedimage.icy'

# Writes the entire encoded image to our file
def writeIcyImage(FILEOUT, bln_verbose = True):
    if bln_verbose:
        def _vprint(what):
            print what
    else:
        def _vprint(what):
            pass

    _vprint('Encoding to iCompress YUV format...')
    f = open(FILEIN, 'r')

    for line in f:
        delimiter = '|'
        delimitedLine = [e+delimiter for e in line.split(delimiter) if e]
        bitstream = delimitedLine[-1][:-1] # Remove a '|' as result of the split
    f.close()
    bitstream_length = len(bitstream)
    BIT_ENCODING_SIZE = 32

    f = open(FILEOUT, 'w')
    # Write codebook as regular
    codebook = np.asarray(delimitedLine[:-1]).flatten()
    _vprint('First 32 bits: ' + str(bitstream[0:32]))
    for i in range(0, len(codebook)):
        f.write(codebook[i])
    for i in range(0, bitstream_length, BIT_ENCODING_SIZE):
        endIndex = BIT_ENCODING_SIZE
        if i + BIT_ENCODING_SIZE > bitstream_length:
            endIndex = bitstream_length
        bits = bitstream[i:i+endIndex]
        int_value = int(bits, base = 2)
        bin_array = struct.pack('I', int_value)
        f.write(bin_array)
    f.close()
    _vprint('Successfully encoded raw to iCompress YUV!')
writeIcyImage(FILEOUT)

## Reading our encoded image
We demonstrate that our encoded image can be easily decoded again to its original encoded bitstream.

In [ ]:
FILEIN = 'encodedimage.icy'

def readIcyImage(FILEIN):
    FILEOUT = FILEIN[:-4] + '.raw'
    marker_count = 0
    f_in = open(FILEIN, 'r')
    f_out = open(FILEOUT, 'w')
    bytestream = ''
    content = f_in.readlines()

    # Read in bytestream
    for line in content:
        bytestream += line.encode('hex')
    print len(bytestream)
    f_test = open('stream', 'w')
    begin_index = 0
    # Convert bytestream to bitstream
    while begin_index < len(bytestream): # We iterate over 2 hexes a time, due to our encoder
        if begin_index + 8 > len(bytestream):
            codes = bytestream[begin_index:].zfill(8)
        else:
            codes = bytestream[begin_index:begin_index+8].zfill(8)
        ascii = codes.decode('hex') # Our codebook is in plain ascii
        # Check when to start also unpacking the binary data
        if marker_count < 6:
            binary = ascii
            if '|' in ascii and marker_count == 5:
                marker_count += 1
                new_end_index = begin_index + 2*ascii.index('|')+2 # we have 4 ascii characters and took 8 characters as hex, index is from 0
                binary = bytestream[begin_index:new_end_index].decode('hex')
                begin_index = new_end_index
            elif '|' in ascii:
                marker_count += 1
                begin_index += 8
            else:
                begin_index += 8
        else:
            orig_int_value = struct.unpack('I', codes.decode('hex'))[0]
            binary = bin(orig_int_value)[2:] # Skip '0b' bit marker
            if len(binary) != 32:
                binary = binary.zfill(32)
            begin_index += 8
        f_out.write(binary)
    f_out.close()
    f_in.close()

# Execute reading in our .icy image to convert to .raw again
readIcyImage(FILEIN)

## Mapping bitstream to the DCTCs
We read the bitstream and bit per bit and map the codes to its original DCTC using the Canonical Huffman codebook.

In [ ]:
def readRaw(FILEIN):
    f_in = open(FILEIN, 'r')
    for line in f_in:
        bitstream = line
    f_in.close()

    return bitstream

def readCodebook(bitstream):
    start_index = 0
    end_index = start_index + 1
    marker_found = 0

    print 'Decoding codebook...'
    # Decode codebook
    while not marker_found == 6:
        bits =  bitstream[start_index:end_index]
        if '|' in bits:
            marker_found += 1
        start_index = end_index
        end_index += 1

    codebook_string = bitstream[0:end_index-1]
    codebook = codebook_string.split('|')[:-1]
    wrapper = []
    for i in range(0, len(codebook), 2):
        freqs = np.asarray(codebook[i].split(','))
        symbols = np.asarray(codebook[i+1].split(','))
        wrapper.append([freqs, symbols])
    wrapper = np.asarray(wrapper)

    canonicalHuffmanCode_decoded = decodeCanonicalCodebook(wrapper, bln_verbose=False)
    lookupTable_decoded = convertToLookupTable(canonicalHuffmanCode_decoded)
    inv_lookupTable_decoded = [{v: k for k, v in table.iteritems()} for table in lookupTable_decoded]

    return start_index, inv_lookupTable_decoded

def decodeImage(bitstream, lookupTable, begin_index):
    all_dctcs = []
    dctcs = []
    prev = 1
    dctc = 1
    channel_lookupTable = lookupTable[0]
    start_index = begin_index
    print 'Decoding Y channel...'
    while start_index <= bitstream_length:
        if int(prev) == 0 and int(dctc) == 0 and len(all_dctcs) == 0:
            dctc = 1
            channel_lookupTable = lookupTable[1]
            all_dctcs.append(dctcs)
            dctcs = []
            print 'Bit index:', start_index
            print 'Decoding Cr channel...'
        elif int(prev) == 0 and int(dctc) == 0 and len(all_dctcs) == 1:
            dctc = 1
            channel_lookupTable = lookupTable[2]
            all_dctcs.append(dctcs)
            dctcs = []
            print 'Bit index:', start_index
            print 'Decoding Cb channel...'
        prev = dctc

        if len(all_dctcs) < 3:
            end_index = start_index + 1
            max_val = max([len(v) for v in channel_lookupTable.keys()])

            while (end_index - start_index) <= max_val:
                bitstring = bitstream[start_index:end_index]
                if bitstring in channel_lookupTable:
                    dctc = channel_lookupTable[bitstring]
                    dctc_index_found = end_index
                end_index += 1
            dctcs.append(dctc)
            start_index = dctc_index_found
    all_dctcs.append(dctcs)
    print 'Done decoding channels'
    print 'Bit index:', start_index
    return all_dctcs

 Decode our bitstream to get our zig-zagged DCTCs

In [ ]:
FILEIN = 'encodedimage.raw'
bitstream = readRaw(FILEIN)
bitstream_length = len(bitstream)
print 'Bitstream length:', bitstream_length, 'bits'

begin_index, inv_lookupTable_decoded = readCodebook(bitstream)
all_dctcs = decodeImage(bitstream, inv_lookupTable_decoded, begin_index)

## Decoded RLE DCTCs
Verify that our decoded RLE DCTCs correspond to the original RLE DCTCs.

In [ ]:
compare = lambda x, y: collections.Counter(x) == collections.Counter(y)

print 'Decoded Y Zig-zag DCTCs equal to original?', compare(map(int, all_dctcs[0]), map(int, zigzag_quant[0]))
print 'Decoded Cr Zig-zag DCTCs equal to original?', compare(map(int, all_dctcs[1]), map(int, zigzag_quant[1]))

# This may happen to be false, but this is due to trailing zeros we needed to add for our decoder to cope with it
print 'Decoded Cb Zig-zag DCTCs equal to original?', compare(map(int, all_dctcs[2]), map(int, zigzag_quant[2]))

## Run length decoding and reverse zig-zag
Now we have obtained our RLE values with zigzag, we have to reverse the process again.

In [ ]:
def fix_dc_difference(quant_table):
    for channel in range(0,len(quant_table)):
        h = 0
        w = 0
        first_el = 0
        if channel == 0:
            h = IMAGE_HEIGHT
            w = IMAGE_WIDTH
        else:
            h = IMAGE_HEIGHT/2
            w = IMAGE_WIDTH/2
        for height_first in range(0,h,MACROBLOCK_SIZE):
            for width_first in range(0,w,MACROBLOCK_SIZE):
                if height_first == 0 and width_first == 0:
                    quant_table[channel][height_first][width_first] = quant_table[channel][height_first][width_first]
                else:
                    quant_table[channel][height_first][width_first] = first_el + quant_table[channel][height_first][width_first]
                first_el = quant_table[channel][height_first][width_first]
    return quant_table

def check_next_pos(index, macroblock_nr,mw,mh, channel):
    if(index == MACROBLOCK_SIZE*MACROBLOCK_SIZE - 1):
        index = 0
        macroblock_nr += 1
        mw += 1
        if((macroblock_nr % (IMAGE_WIDTH/MACROBLOCK_SIZE) == 0 and channel == 0) or (macroblock_nr % (IMAGE_WIDTH/MACROBLOCK_SIZE/2) == 0 and channel != 0) ):
            mw = 0
            mh += 1
    else:
        index += 1
    return index, macroblock_nr,mw,mh

def decode_zigzag_pair(quant_arr_pair, quant_table, index, macroblock_nr, channel):
    if channel == 0 :
        macroblock_width = macroblock_nr % (IMAGE_WIDTH/MACROBLOCK_SIZE)
        macroblock_height = math.floor(macroblock_nr / (IMAGE_WIDTH/MACROBLOCK_SIZE))
    else:
        macroblock_width = macroblock_nr % (IMAGE_WIDTH/MACROBLOCK_SIZE/2)
        macroblock_height = math.floor(macroblock_nr / (IMAGE_WIDTH/MACROBLOCK_SIZE/2))
    for add_zeros in range(int(quant_arr_pair[0])):
        quant_table[int(macroblock_height*MACROBLOCK_SIZE + i_zig[index])][int(macroblock_width*MACROBLOCK_SIZE + j_zig[index])] = 0
        index,macroblock_nr,macroblock_width,macroblock_height = check_next_pos(index, macroblock_nr, macroblock_width,macroblock_height, channel)
    quant_table[int(macroblock_height*MACROBLOCK_SIZE + i_zig[index])][int(macroblock_width*MACROBLOCK_SIZE + j_zig[index])] = quant_arr_pair[1]
    index,macroblock_nr,macroblock_width,macroblock_height = check_next_pos(index, macroblock_nr, macroblock_width,macroblock_height, channel)
    return quant_table, index, macroblock_nr

def decode_zigzag(decode_quant_arr):
    normal_quant = []
    trailing_zeros = 0
    for channel in range(0,len(decode_quant_arr)):
        current_index = 0
        current_macroblock = 0
        if channel == 0:
            normal_quant.append(np.zeros((IMAGE_HEIGHT, IMAGE_WIDTH)))
        else:
            normal_quant.append(np.zeros((IMAGE_HEIGHT/2, IMAGE_WIDTH/2)))
        for i in range(0, len(decode_quant_arr[channel]), 2):
            normal_quant[channel], current_index, current_macroblock = decode_zigzag_pair(decode_quant_arr[channel][i:i+2],normal_quant[channel], current_index, current_macroblock, channel)
    normal_quant = fix_dc_difference(normal_quant) #With DC difference
    return normal_quant

decoded_zigzag = decode_zigzag(zigzag_quant) # Decode zig-zag

## Decoded DCTCs
We once more verify that our decoding of the RLE is performed successfully.
We compare the decoded results to the original DCTCs.

In [ ]:
# Verify that our decoded DCTCs are completely equal to the DCTCs resulting from the quantization of the image.
compare = lambda x, y: collections.Counter(x) == collections.Counter(y)

print 'Decoded Y DCTCs equal to original?', compare(map(int, list(decoded_zigzag[0].flatten())), map(int, list(image_YCrCb_dct_quantized[0].flatten())))
print 'Decoded Cr DCTCs equal to original?', compare(map(int, list(decoded_zigzag[1].flatten())), map(int, list(image_YCrCb_dct_quantized[1].flatten())))
print 'Decoded Cb DCTCs equal to original?', compare(map(int, list(decoded_zigzag[2].flatten())), map(int, list(image_YCrCb_dct_quantized[2].flatten())))

In [ ]:
# Taken from the MMSR systems lab 1.
def decode_quantized_dct_block(macroblock_dct_quantized, quantization_table):
    """
    Decode quantized DCT coefficients given the quantization table used at the encoding step.

    This function is dual to quantizemacroblock_dct().
    DCT coefficients can be negative, we need to map the macroblock values in the range [0, 255].
    """
    macroblock_dct_quantized = np.multiply(macroblock_dct_quantized, quantization_table)

    macroblock_decoded = cv2.idct(macroblock_dct_quantized)
    return np.clip(macroblock_decoded, 0, 255).astype(np.uint8)

In [ ]:
# Taken from the MMSR systems lab 1 with several modifications.
def decode_image_YCrCb_dct_quantized(image_YCrCb_dct_quantized, scaled_quantization_tables,
                                     image_size, block_size = MACROBLOCK_SIZE, bln_verbose = True):
    if bln_verbose:
        def _vprint(what):
            print what
    else:
        def _vprint(what):
            pass
    _vprint('Decoding quantized DCT')
    pass

    image_height = image_size[0]
    image_width = image_size[1]
    number_of_channels = image_size[2]
    _vprint(number_of_channels)

    image_YCrCb_decoded = np.zeros(image_size, dtype=np.uint8)
    _vprint('. decoded image size: %d x %d' % (image_width, image_height))

    scaled_fdm = computeSelectionMacroBlock(inverted_fdm/float(np.max(inverted_fdm)))

    for channel_index, (channel_dct_quantized, quantization_table) in enumerate(zip(image_YCrCb_dct_quantized, scaled_quantization_tables)):
        # Channel properties.
        channel_size = np.shape(channel_dct_quantized)
        number_of_horizontal_macroblocks, number_of_vertical_macroblocks = (get_number_of_macroblocks(channel_size, block_size))

        _vprint('. channel #%d' % channel_index)
        _vprint('  channel size: %s' % format_image_size(channel_size))
        _vprint('  number of macroblocks: %d x %d' % (number_of_horizontal_macroblocks, number_of_vertical_macroblocks))

        # Initialize the decoded channel matrix.
        decoded_channel = np.zeros(channel_size, dtype=channel_dct_quantized.dtype)

        # Our boolean matrix to indicate which macroblocks should be quantized.
        macroblock_selector = channelRegions[channel_index]

        # Each macroblock is independently decoded.
        for row in range(number_of_vertical_macroblocks):
            for column in range(number_of_horizontal_macroblocks):
                # Extract the macroblock from the quantized DCT coefficients matrix.
                (left_top, right_bottom) = macroblock_coords_to_pixel_vertices((column, row), block_size)
                macroblock_dct_quantized = channel_dct_quantized[
                    left_top[1]:right_bottom[1]+1, left_top[0]:right_bottom[0]+1]

                if macroblock_selector[column, row] == 1:
                    # Get the FDM scalar
                    FDM_scalar = scaled_fdm[column, row]

                    # Decode quantized DCT coefficients.
                    macroblock_decoded = decode_quantized_dct_block(
                        macroblock_dct_quantized, FDM_scalar*quantization_table)
                else:
                    macroblock_decoded = macroblock_dct_quantized

                # Copy macroblock decoded data.
                decoded_channel[
                    left_top[1]:right_bottom[1]+1, left_top[0]:right_bottom[0]+1] = macroblock_decoded

        # Resize the decoded channel matrix.
        image_YCrCb_decoded[:, :, channel_index] = cv2.resize(decoded_channel, (image_width, image_height))

    return image_YCrCb_decoded

## Decoding of the quantized DCTCs of the <code>.icy</code> image
We have now successfully decoded our <code>.icy</code> file and reconstructed our original DCTCs. This means we are ready to decode the quantized image. <br />
Moreover, we can now also see what our compressed image looks like.

In [ ]:
# Now, convertYUV to RGB and compare the original and the compressed image.
image_YCrCb_decoded = decode_image_YCrCb_dct_quantized(
    decoded_zigzag, quantization_tables, im_size)
image_BGR_decoded = cv2.cvtColor(image_YCrCb_decoded, cv2.cv.CV_YCrCb2BGR)

ipynb_show_cv2_image(im, 'original')
ipynb_show_cv2_image(image_BGR_decoded, '.icy compressed')

## Computing fixation density maps from eye fixation data
This section will handle the computation of fixation density maps (FDMs) from the eye fixation data. The eye fixation data was retrieved from the following location: http://ivc.univ-nantes.fr/en/databases/HD_UHD_Eyetracking_Videos/.

An FDM will be computed for each frame seperately. The functions for doing this are listed first, after which the code to execute them is listed.

First load the raw fixation data and subsample it to enable more efficient processing. The FDMs will be supersampled back to the video frame size at the end. This does not cause a large loss of quality, because the FDMs don't contain much detail

In [ ]:
# Input: file_name to read fixations from as a string
# Output: List of tuples, with as first element tuple of start and end time of fixation,
# and as second element tuple of x and y position of fixation.
def read_fixations(file_name):
    fixations = []
    with open(file_name, 'rb') as f:
        csv_reader = csv.reader(f, 'excel-tab')
        next(csv_reader)
        for fixation in csv_reader:
            fixations.append(((float(fixation[0]), float(fixation[1])),(int(float(fixation[2])), int(float(fixation[3])))))
    return fixations

# Input: raw fixations and a subsampling ratio
# Output: A subsampled set of fixations
def subsample_fixations(fixations, ratio):
    fixations_subsampled = []
    for fixation in fixations:
        fixations_subsampled.append((fixation[0], (fixation[1][0] / ratio, fixation[1][1] / ratio)))

    return fixations_subsampled

Then, convert the time-based fixations to frame-based fixations. This is necessary to find out which frames a fixation belongs to. `time_fixations_to_frame_fixations()` handles a list of fixations, `time_fixation_to_frame_fixations()` handles one fixation at a time.

In [ ]:
# Input: time and frames per second for the video
# Ouput: zero-based frame number corresponding to the given time
def time_to_frame_number(time, fps):
    return max(int(time / 1000.0 * fps), 0)

# Input: one fixationn occurrence with the time it occurred at
# Output: the input fixation for each frame it occurred in
def time_fixation_to_frame_fixations(time_fixation, fps):
    frame_fixations = []
    fixation_time = time_fixation[0]
    start_frame = time_to_frame_number(fixation_time[0], fps)
    end_frame = time_to_frame_number(fixation_time[1], fps)
    for frame_number in range(start_frame, end_frame + 1):
        frame_fixations.append((frame_number,time_fixation[1]))

    return frame_fixations

# Input: list of fixations indicated by time they occurred.
# Output: list of fixations indicated by frame they occurred in.
def time_fixations_to_frame_fixations(time_fixations, fps):
    frame_fixations = []
    for fixation in time_fixations:
        frame_fixations.extend(time_fixation_to_frame_fixations(fixation, fps))
    return frame_fixations

After that, organize the fixations in a dictionary to be able to access them efficiently per frame.

In [ ]:
# Input: list of fixations indicated by frame number
# Output: ditionary with as key frame number, as value list of fixations
def fixation_list_to_dict(fixation_list):
    fixation_dict = {}
    for fixation in fixation_list:
        if fixation[0] not in fixation_dict:
            fixation_dict[fixation[0]] = []
    for fixation in fixation_list:
        fixation_dict[fixation[0]].append(fixation[1])
    return fixation_dict

For each frame, compute the density map using a gaussian function around the original fixation points.

In [ ]:
# Computes gaussian value in 2d space using the given parameters.
def gaussian_2d(x, y, x_base, y_base, amplitude = 1, spread = 1):
    x_part = float(((x-x_base)**2))/float((2*spread**2))
    y_part = float(((y-y_base)**2))/float((2*spread**2))
    return amplitude * np.exp(-(x_part + y_part))


# Input: List of fixations for one frame
# Output: Fixation density map for one frame
def fixations_to_fdm(fixations, frame_size, fixation_spread=0.1):
    fdm = np.zeros(frame_size)
    average_frame_size = (frame_size[0] + frame_size[1]) / 2.0
    for fixation in fixations:
        for i in range(0, frame_size[0]):
            for j in range(0, frame_size[1]):
                fdm[i][j] += gaussian_2d(j, i, fixation[0], fixation[1], 1, 0.05 * average_frame_size)
    return fdm

Now normalize each FDM and convert it to uint8 to prepare it for compression by OpenCV to save space.

In [ ]:
def fdm_max_value(fdm):
    max_value = 0
    for i in range(fdm.shape[0]):
        max_value = max(max_value, max(fdm[i]))
    return max_value

# Input: fdm that has not been normalized
# Output: fdm that has been normalized to values between 0 and 1.
def normalize_fdm(fdm):
    max_value = fdm_max_value(fdm)
    normalized_fdm = np.zeros(fdm.shape)
    for i in range(fdm.shape[0]):
        for j in range(fdm.shape[1]):
            normalized_fdm[i][j] = fdm[i][j] / max_value
    return np.array(normalized_fdm * 255, dtype='uint8')

Now supersample each FDM back to the original frame size, and encode it to PNG to save space. Add a decode function too.

In [ ]:
# Input: original fdm and frame_size to sample to
# Output: supersampled fdm
def supersample_fdm(fdm, frame_size):
    return cv2.resize(fdm, frame_size, interpolation=cv2.INTER_CUBIC)

# Input: fdm
# Output: fdm encoded using png
def encode_fdm(fdm):
    return cv2.imencode('.png', fdm, (cv2.IMWRITE_PNG_COMPRESSION, 9))[1]

# Input: fdm encoded by encode_fdm()
# Output: decoded fdm
def decode_fdm(fdm):
    return np.array(cv2.imdecode(fdm, cv2.CV_LOAD_IMAGE_GRAYSCALE) / 255.0, dtype='float32')

Finally add a function that uses all previous parts of computing the FDMs to implement the whole pipeline. Also add a function to show the final FDMs as images.

In [ ]:
def compute_fdms_all_frames(file_name, fps, frame_size, subsample_ratio):
    print('Processing raw data...')
    raw_fixations = read_fixations(file_name)
    subsampled_fixations = subsample_fixations(raw_fixations, subsample_ratio)
    frame_fixations = time_fixations_to_frame_fixations(subsampled_fixations, fps)
    fixation_dict = fixation_list_to_dict(frame_fixations)
    subsampled_frame_size = (frame_size[0] / subsample_ratio + 1, frame_size[1] / subsample_ratio + 1)

    final_fdms = {}
    counter = 0
    for fixations in fixation_dict:
        print('Computing fdm for frame ' + str(counter) + '...')
        counter += 1
        fdm = fixations_to_fdm(fixation_dict[fixations], subsampled_frame_size, 0.05)
        normalized_fdm = normalize_fdm(fdm)
        supersampled_fdm = supersample_fdm(normalized_fdm, (FRAME_SIZE[1], FRAME_SIZE[0]))
        final_fdms[fixations] = encode_fdm(supersampled_fdm)
    return final_fdms

def show_fdms(fdms):
    fdm_keys = sorted(fdms.iterkeys())
    for fdm in fdm_keys:
        ipynb_show_image(decode_fdm(fdms[fdm]), 'Frame ' + str(fdm), _DEFAULT_IMAGE_FIGSIZE, True)

# Only shows a few examples to save time
def show_fdms_examples(fdms):
    ipynb_show_image(decode_fdm(fdms[0]), 'Frame 0', _DEFAULT_IMAGE_FIGSIZE, True)
    ipynb_show_image(decode_fdm(fdms[50]), 'Frame 50', _DEFAULT_IMAGE_FIGSIZE, True)
    ipynb_show_image(decode_fdm(fdms[100]), 'Frame 100', _DEFAULT_IMAGE_FIGSIZE, True)
    ipynb_show_image(decode_fdm(fdms[150]), 'Frame 150', _DEFAULT_IMAGE_FIGSIZE, True)
    ipynb_show_image(decode_fdm(fdms[200]), 'Frame 200', _DEFAULT_IMAGE_FIGSIZE, True)
    ipynb_show_image(decode_fdm(fdms[250]), 'Frame 250', _DEFAULT_IMAGE_FIGSIZE, True)
    ipynb_show_image(decode_fdm(fdms[297]), 'Frame 297', _DEFAULT_IMAGE_FIGSIZE, True)

Now execute the `compute_fdms_all_frames()` function on an example video.

In [ ]:
FPS = 25
FRAME_SIZE = (2160, 3840)
SUBSAMPLE_RATIO = 288
FILE_NAME = "fixations/BBB_seq1_s3840x2160p25n300v0_Fixations.csv"

fdms = compute_fdms_all_frames(FILE_NAME, FPS, FRAME_SIZE, SUBSAMPLE_RATIO)

And last, show the result as an image. Only some examples are shown. To show the FDM for every frame, use the `show_fdms()` function.

In [ ]:
show_fdms_examples(fdms)

## Compress a video
Now we have all the FDMs generated for our video, we will start testing our compression method on the video.
These helper functions below will help us convert the .YUV file format into an easier iterable format.

In [ ]:
# From a YUV444 format, return the Y, U, V as separate components in YUV420 format.
def YUV444toYUV422(image_YCrCb_decoded, image_height,image_width):
    store_Y_decoded_image = np.chararray(image_height * image_width)
    store_Cr_decoded_image = np.chararray(image_height * image_width / 4)
    store_Cb_decoded_image = np.chararray(image_height * image_width / 4)
    print image_YCrCb_decoded.shape
    for i in range(0,image_YCrCb_decoded.shape[0]):
        for j in range(0,image_YCrCb_decoded.shape[1]):
            store_Y_decoded_image[i*image_width+j] = chr(image_YCrCb_decoded[i][j][0])
            if(i % 2 == 0 and j % 2 ==0 ):
                store_Cr_decoded_image[i/2*image_width/2+j/2] = chr(image_YCrCb_decoded[i][j][1])
                store_Cb_decoded_image[i/2*image_width/2+j/2] = chr(image_YCrCb_decoded[i][j][2])
    return store_Y_decoded_image, store_Cr_decoded_image, store_Cb_decoded_image

# Play a YUV420 video
def YUV2video(filename):
    ffmpeg_bin = 'ffmpeg'
    command = [ffmpeg_bin,
               '-f', 'rawvideo',
               '-r',str(FPS),
               '-video_size', str(FRAME_SIZE[1])+'x'+str(FRAME_SIZE[0]),
               '-pix_fmt', 'yuv420p',
               '-i', filename,
               '-y',
               '-c:v', 'libx264', #H264 encoding
               '-preset', 'ultrafast', '-qp', '0', './ultraeye/output.mp4' #Output
              ]

    try:
        sp.check_output(command,stderr=sp.STDOUT)
    except sp.CalledProcessError as e:
        raise RuntimeError("command '{}' return with error (code {}): {}".format(e.cmd, e.returncode, e.output))

## Converting every frame
Here we loop over all the frames and one by one will run all the steps for compressing and decompressing our frames.

In [ ]:
FILE_NAME_VIDEO = 'video/BBB_seq1_s3840x2160p25n300v0.yuv'

# from shutil import copyfile
# FILE_NAME_VIDEO_COPY = 'ultraeye/BBB_seq1_s3840x2160p25n300v0_copy.yuv'
# copyfile(FILE_NAME_VIDEO, FILE_NAME_VIDEO_COPY)
# FILE_NAME_VIDEO = FILE_NAME_VIDEO_COPY

MAX_PIX_VAL = 255

# Reference for reading input: https://raspberrypi.stackexchange.com/questions/28033/reading-frames-of-uncompressed-yuv-video-file
with open(FILE_NAME_VIDEO, 'rb+') as stream:
    Framecount = 0 #The frame it is currently processing
    while Framecount != 300: # Amount of frames which will be compressed
        print 'Processing frame %d...' % (Framecount)
        fwidth = FRAME_SIZE[1]
        fheight = FRAME_SIZE[0]
        stream.seek(Framecount * fheight * fwidth * 1.5) # Step size of one frame (YUV420 format)
        Y = np.fromfile(stream, dtype=np.uint8, count=fwidth*fheight).\
                reshape((fheight, fwidth))
        # Load the UV (chrominance) data from the stream, and double its size
        U = np.fromfile(stream, dtype=np.uint8, count=(fwidth//2)*(fheight//2)).\
                reshape((fheight//2, fwidth//2)).\
                repeat(2, axis=0).repeat(2, axis=1)
        V = np.fromfile(stream, dtype=np.uint8, count=(fwidth//2)*(fheight//2)).\
                reshape((fheight//2, fwidth//2)).\
                repeat(2, axis=0).repeat(2, axis=1)
        # Stack the YUV channels together, crop the actual resolution, convert to
        # floating point for later calculations, and apply the standard biases
        YUV = np.dstack((Y, U, V))[:fheight, :fwidth, :].astype(np.uint8)
        im = YUV
        im = cv2.cvtColor(im, cv2.cv.CV_YCrCb2BGR)
        im_size = np.shape(im) #don't think this ever changes
        ##Load new density map here
        im_fdm = (decode_fdm(fdms[Framecount]) * 255).astype(np.uint8)
        white_threshold = THRESHOLD_SCALAR*MAX_PIX_VAL
        inverted_fdm = cv2.bitwise_not(im_fdm)
        _, im_fdm_thresh = cv2.threshold(im_fdm, white_threshold, MAX_PIX_VAL, cv2.THRESH_BINARY)
        inverted_fdm_thres = cv2.bitwise_not(im_fdm_thresh)
        channelRegions = computeChannelRegions(inverted_fdm_thres)


        ## Encode image
        image_YCrCb_subsampled = subsample_chrominance_channels(im)
        image_YCrCb_dct_quantized = quantize_image_YCrCb_dct(
        image_YCrCb_subsampled, quantization_tables, im_size)

        # Decoding image
        image_YCrCb_decoded = decode_image_YCrCb_dct_quantized(
            image_YCrCb_dct_quantized, quantization_tables, im_size)

        #Write back to file
        y_dec, u_dec, v_dec = YUV444toYUV422(image_YCrCb_decoded, FRAME_SIZE[0],FRAME_SIZE[1])
        stream.seek(Framecount * fheight * fwidth * 1.5) # For Move back.
        stream.write(y_dec) #Store as YUV4:2:0
        stream.write(u_dec)
        stream.write(v_dec)
        Framecount += 1
YUV2video(FILE_NAME_VIDEO)

## Output frame in video
Helper function to check if frame in video seems fine.

In [ ]:
ffmpeg_bin = 'ffmpeg'
command = [ffmpeg_bin,
           '-f', 'rawvideo',
           '-r','300',
           '-video_size', '3840x2160',
           '-pix_fmt', 'yuv420p',
           '-i', FILE_NAME_VIDEO,
           '-vf', 'select=gte(n\,2)', #Frame
           #'-qscale', '0', #No reduction
           '-frames:v', '1','-y',  #Amount of frame
           '-f', 'image2pipe',
           '-pix_fmt', 'yuv420p',
           '-c:v', 'rawvideo',
           '-'
          ]
proc = sp.Popen(command, stdout=sp.PIPE, stderr=sp.PIPE)
output, errors = proc.communicate()
print output[0:100]

## Evaluation
For the evaluation, we will compare the performance of our algorithm against the JPEG algorithm. We will compare the algorithms based on these two properties:
- Quality of image after compression, compared to the base image. This will be done using the structural similarity measure.
- Compression ratio

### Quality of image
For comparing the quality of the final image, we will use the SSIM measure. The SSIM measure is the structural similarity measure that measures the similarity of two images based on how a human would compare them. Applying it to two images results in a value from 0 to 1, where a higher value indicates more similarity.

We will compare both the iCompress and JPEG version of the image to the original image using the SSIM measure.

In [ ]:
ssim_icompress = skimage.measure.compare_ssim(im, image_BGR_decoded, multichannel=True, gaussian_weights=True)
print "SSIM iCompress = %.6f" % (ssim_icompress)

im_jpeg_encoded = cv2.imencode('.jpg', im, (cv2.IMWRITE_JPEG_QUALITY, 24))[1]
im_jpeg_decoded = cv2.imdecode(im_jpeg_encoded, cv2.CV_LOAD_IMAGE_COLOR)

ssim_jpeg = skimage.measure.compare_ssim(im, im_jpeg_decoded, multichannel=True, gaussian_weights=True)
print "SSIM JPEG = %.6f" % (ssim_jpeg)

As can be seen from the output, the overall quality of the iCompress algorithm with the current settings is about the same as the JPEG algorithm at quality level 24.

Let us now visualize the differences by showing the original, iCrompress compressed, and JPEG compressed images side by side.

In [ ]:
ipynb_show_cv2_image(im, 'original')
ipynb_show_cv2_image(image_BGR_decoded, 'iCompress')
ipynb_show_cv2_image(im_jpeg_decoded, 'JPEG quality 24')

The difference between iCompress and JPEG is most apparent in the sky: it is of much lower quality in the iCompress case.

Let us now zoom in to an area that has been left mostly untouched by iCompress because of the povided fixation density map.

In [ ]:
vertical_slice = slice(750, 2250)
horizontal_slice = slice(1750, 2750)
ipynb_show_cv2_image(im[vertical_slice, horizontal_slice, :], 'original zoomed')
ipynb_show_cv2_image(image_BGR_decoded[vertical_slice, horizontal_slice, :], 'iCompress zoomed')
ipynb_show_cv2_image(im_jpeg_decoded[vertical_slice, horizontal_slice, :], 'JPEG quality 24 zoomed')

From the above images, one can see that colored houses on the left of the street and the church are of higher quality in the iCompress case than in the JPEG case. In fact, the iCompress case is similar in quality as expected to the original. The artifacts in the clouds in the iCompress image are even more apparent at this zoom level, though.

Let us now zoom in more closely to the colored houses to see the difference more clearly.

In [ ]:
vertical_slice = slice(1250, 1750)
horizontal_slice = slice(1750, 2350)
ipynb_show_cv2_image(im[vertical_slice, horizontal_slice, :], 'original zoomed even more')
ipynb_show_cv2_image(image_BGR_decoded[vertical_slice, horizontal_slice, :], 'iCompress zoomed even more')
ipynb_show_cv2_image(im_jpeg_decoded[vertical_slice, horizontal_slice, :], 'JPEG quality 24 zoomed even more')

In this example the houses are clearly of better quality in the iCompress image.

### Compression ratio

Now we are going to look at the difference in compression ratio between iCompress and JPEG. To do this, we have to retrieve the actual file size of the files on disk that have been compressed by the two algrorihms.

The iCompress file has already been written to disk earlier in the notebook. Let's now also write a JPEG file to disk and retrieve it's size. We also retrieve the size of the iCompress file.

First we set the JPEG quality to the level at which the quality of iCompress and JPEG is the same.

In [ ]:
def bytes_to_MB(bytes):
    return bytes/float(1024**2)

print 'Size of original file: %.3fMB' % bytes_to_MB(os.path.getsize('ultraeye/C01_UHD.tif'))
print 'Size of iCompress file: %.3fMB' % bytes_to_MB(os.path.getsize('encodedimage.icy'))

cv2.imwrite('im_jpeg_encoded.jpg', im, (cv2.IMWRITE_JPEG_QUALITY, 24))
print 'Size of JPEG file: %.3fMB' % bytes_to_MB(os.path.getsize('im_jpeg_encoded.jpg'))

Here it is quite clear that the JPEG algorithm's compression ratio is much better than iCompress's; actually it is an order of magnitude better. This is probably due to JPEG having much more efficient coding for storage. Though note that the iCompress file is still also an order of magnitude smaller than the original tif file.

To be more precise, for this case the compression ratios are as follows:
- iCompress: 0.118
- JPEG: 0.011

Now let's see what happens to the image quality differences when we set the JPEG quality so that the file sizes are about the same. We will use the same street part of the picture as above.

In [ ]:
print 'Size of iCompress file: %.3fMB' % bytes_to_MB(os.path.getsize('encodedimage.icy'))

cv2.imwrite('im_jpeg_encoded.jpg', im, (cv2.IMWRITE_JPEG_QUALITY, 98))
print 'Size of JPEG file: %.3fMB' % bytes_to_MB(os.path.getsize('im_jpeg_encoded.jpg'))

In [ ]:
print "SSIM iCompress = %.6f" % (ssim_icompress)

im_jpeg_encoded_high_quality = cv2.imencode('.jpg', im, (cv2.IMWRITE_JPEG_QUALITY, 98))[1]
im_jpeg_decoded_high_quality = cv2.imdecode(im_jpeg_encoded_high_quality, cv2.CV_LOAD_IMAGE_COLOR)

ssim_jpeg_high_quality = skimage.measure.compare_ssim(im, im_jpeg_decoded_high_quality, multichannel=True, gaussian_weights=True)
print "SSIM JPEG = %.6f" % (ssim_jpeg_high_quality)

vertical_slice = slice(1250, 1750)
horizontal_slice = slice(1750, 2350)
ipynb_show_cv2_image(im[vertical_slice, horizontal_slice, :], 'original zoomed even more')
ipynb_show_cv2_image(image_BGR_decoded[vertical_slice, horizontal_slice, :], 'iCompress zoomed even more')
ipynb_show_cv2_image(im_jpeg_decoded_high_quality[vertical_slice, horizontal_slice, :], 'JPEG quality 24 zoomed even more')

Now almost no differences can be seen in quality for the colored buildings on the left side of the street when comparing iCompress and JPEG. However, in the parts where iCompress is not of high quality, JPEG is clearly better.

We could try zooming in close to the pixel level to see if the high quality parts of the iCompress image are still better than the JPEG's.

In [ ]:
vertical_slice = slice(1475, 1725)
horizontal_slice = slice(1950, 2150)
ipynb_show_cv2_image(im[vertical_slice, horizontal_slice, :], 'original extreme zoom')
ipynb_show_cv2_image(image_BGR_decoded[vertical_slice, horizontal_slice, :], 'iCompress extreme zoom')
ipynb_show_cv2_image(im_jpeg_decoded_high_quality[vertical_slice, horizontal_slice, :], 'JPEG quality 24 extreme zoom')

There is no difference to be seen here, even not between the original and the two compressed versions.

## Quality and compression ratio summary
In summary, we have seen that the iCompress compression algorithm can deliver better quality than JPEG in its high quality areas when the overall quality level is the same between the two algorithms. This goes at the expense, and quite clearly so, of the lower quality areas, which can suffer quite heavily.

The JPEG algorithm is clearly better at getting good quality with a small file size. The files of iCompress are a lot bigger for the same quality. If one chooses the file size to be about the same, the JPEG image will outperform the iCompress image strongly in the low-quality areas of the iCompress image. The quality is about the same in high-quality areas of the iCompress image, in that case.